### This exercise is designed to pair with [this tutorial](https://www.kaggle.com/rtatman/bigquery-machine-learning-tutorial). If you haven't taken a look at it yet, head over and check it out first. (Otherwise these exercises will be pretty confusing!) -- Rachael 

# Stocking rental bikes

![bike rentals](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Bay_Area_Bike_Share_launch_in_San_Jose_CA.jpg/640px-Bay_Area_Bike_Share_launch_in_San_Jose_CA.jpg)

You stock bikes for a bike rental company in Austin, ensuring stations have enough bikes for all their riders. You decide to build a model to predict how many riders will start from each station during each hour, capturing patterns in seasonality, time of day, day of the week, etc.

To get started, create a project in GCP and connect to it by running the code cell below. Make sure you have connected the kernel to your GCP account in Settings.

In [1]:
# Set your own project id here
PROJECT_ID = "kaggle-bqml-camp" # a string, like 'kaggle-bigquery-240818'

from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID, location="US")
dataset = client.create_dataset('model_dataset', exists_ok=True)

from google.cloud.bigquery import magics
from kaggle.gcp import KaggleKernelCredentials
magics.context.credentials = KaggleKernelCredentials()
magics.context.project = PROJECT_ID

In [2]:
%load_ext google.cloud.bigquery

## Linear Regression

Your dataset is quite large. BigQuery is especially efficient with large datasets, so you'll use BigQuery-ML (called BQML) to build your model. BQML uses a "linear regression" model when predicting numeric outcomes, like the number of riders.

## 1) Training vs testing

You'll want to test your model on data it hasn't seen before (for reasons described in the [Intro to Machine Learning Micro-Course](https://www.kaggle.com/learn/intro-to-machine-learning). What do you think is a good approach to splitting the data? What data should we use to train, what data should we use for test the model?

In [3]:
# Initially we want training and test set to be completely different, so for training set we can take data from a previous time period than the period of test set's data


## Training data

First, you'll write a query to get the data for model-building. You can use the public Austin bike share dataset from the `bigquery-public-data.austin_bikeshare.bikeshare_trips` table. You predict the number of rides based on the station where the trip starts and the hour when the trip started. Use the `TIMESTAMP_TRUNC` function to truncate the start time to the hour.

## 2) Exercise: Query the training data

Write the query to retrieve your training data. The fields should be:
1. The start_station_name
2. A time trips start, to the nearest hour. Get this with `TIMESTAMP_TRUNC(start_time, HOUR) as start_hour`
3. The number of rides starting at the station during the hour. Call this `num_rides`.
Select only the data before 2018-01-01 (so we can save data from 2018 as testing data.)

Write your query below:

In [4]:
%%bigquery dataframe_name
SELECT start_station_name,
       TIMESTAMP_TRUNC(start_time, HOUR) as start_hour,
       COUNT(bikeid) as num_rides
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_time < "2018-01-01"
GROUP BY start_station_name, start_hour

You'll want to inspect your data to ensure it looks like what you expect. Run the line below to get a quick view of the data, and feel free to explore it more if you'd like (if you don't know how to do that, the [Pandas micro-course](https://www.kaggle.com/learn/pandas)) might be helpful.

In [5]:
dataframe_name.head(5)

,start_station_name,start_hour,num_rides
0,Zilker Park West,2015-10-02 21:00:00+00:00,88
1,Zilker Park West,2015-10-02 20:00:00+00:00,23
2,Zilker Park West,2015-10-03 21:00:00+00:00,78
3,Nueces @ 3rd,2015-10-03 12:00:00+00:00,11
4,Zilker Park West,2015-10-03 16:00:00+00:00,12


## Model creation

Now it's time to turn this data into a model. You'll use the `CREATE MODEL` statement that has a structure like: 

```sql
CREATE OR REPLACE MODEL`model_dataset.bike_trips`
OPTIONS(model_type='linear_reg') AS 
-- training data query goes here
SELECT ...
    column_with_labels AS label
    column_with_data_1 
    column_with_data_2
FROM ... 
WHERE ... (Optional)
GROUP BY ... (Optional)
```

The `model_type` and `optimize_strategy` shown here are good parameters to use in general for predicting numeric outcomes with BQML.

**Tip:** Using ```CREATE OR REPLACE MODEL``` rather than just ```CREATE MODEL``` ensures you don't get an error if you want to run this command again without first deleting the model you've created.

## 3) Exercise: Create and train the model

Below, write your query to create and train a linear regression model on the training data.

Write your query below:

In [6]:
%%bigquery

CREATE OR REPLACE MODEL `model_dataset.bike_trips`
OPTIONS(model_type='linear_reg') AS
SELECT COUNT(bikeid) as label,
       start_station_name,
       TIMESTAMP_TRUNC(start_time, HOUR) as start_hour
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_time < "2018-01-01"
GROUP BY start_station_name, start_hour

""


## 4) Exercise: Model evaluation

Now that you have a model, evaluate it's performance on data from 2018. 


> Note that the ML.EVALUATE function will return different metrics depending on what's appropriate for your specific model. You can just use the regular ML.EVALUATE funciton here. (ROC curves are generally used to evaluate binary problems, not linear regression, so there's no reason to plot one here.)

Write your query below:

In [7]:
%%bigquery

SELECT *
FROM
ML.EVALUATE(MODEL `model_dataset.bike_trips`, (
SELECT COUNT(bikeid) as label,
       start_station_name,
       TIMESTAMP_TRUNC(start_time, HOUR) as start_hour
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_time >= "2018-01-01"
GROUP BY start_station_name, start_hour
))

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,9.939943,113.813323,1.253446,10.710454,-8.174866,-5.211926


You should see that the r^2 score here is negative. Negative values indicate that the model is worse than just predicting the mean rides for each example.

## 5) Theories for poor performance

Why would your model be doing worse than making the most simple prediction based on historical data?

In [8]:
# Maybe the data are not updated, something has changed

## 6) Exercise: Looking at predictions

A good way to figure out where your model is going wrong is to look closer at a small set of predictions. Use your model to predict the number of rides for the 22nd & Pearl station in 2018. Compare the mean values of predicted vs actual riders.

Write your query below:

In [9]:
%%bigquery

SELECT AVG(ROUND(predicted_label)) as predicted_avg_riders,
       AVG(label) as actual_avg_riders
FROM
ML.PREDICT(MODEL `model_dataset.bike_trips`, (
SELECT COUNT(bikeid) as label,
       start_station_name,
       TIMESTAMP_TRUNC(start_time, HOUR) as start_hour
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_time >= "2018-01-01" AND start_time < "2019-01-01" AND start_station_name ="22nd & Pearl"
GROUP BY start_station_name, start_hour
))

,predicted_avg_riders,actual_avg_riders
0,9.397471,3.954864


What you should see here is that the model is underestimating the number of rides by quite a bit. 

## 7) Exercise: Average daily rides per station

Either something is wrong with the model or something surprising is happening in the 2018 data. 

What could be happening in the data? Write a query to get the average number of riders per station for each year in the dataset and order by the year so you can see the trend. You can use the `EXTRACT` method to get the day and year from the start time timestamp. (You can read up on EXTRACT [in this lesson in the Intro to SQL course](https://www.kaggle.com/dansbecker/order-by)). 

Write your query below:

In [10]:
%%bigquery

WITH rides AS (
    SELECT COUNT(bikeid) AS num_rides,
           start_station_name,
           EXTRACT(DAYOFYEAR from start_time) AS day_of_year,
           EXTRACT(YEAR from start_time) AS year
    FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
    GROUP BY start_station_name, day_of_year,year
    ORDER BY year
),
station_avg AS (
    SELECT AVG(num_rides) AS avg_riders, start_station_name, year
    FROM rides
    GROUP BY start_station_name, year)

SELECT AVG(avg_riders) AS rides_per_station, year
FROM station_avg
GROUP BY year
ORDER BY year

,rides_per_station,year
0,12.115702,2013
1,10.360497,2014
2,12.495171,2015
3,12.459424,2016
4,9.768301,2017
5,15.194780,2018
6,6.045254,2019


## 8) What do your results tell you?

Given the daily average riders per station over the years, does it make sense that the model is failing?

In [11]:
## In 2018 riders per station were much more than in 2019, that's why the model is underestimating the number of rides

# 9) Next steps

Given what you've learned, what improvements do you think you could make to your model? Share your ideas on the [Kaggle Learn Forums](https://www.kaggle.com/learn-forum)! (I'll pick a couple of my favorite ideas & send the folks who shared them a Kaggle t-shirt. :)